In [ ]:

# GET THE ROW NUMBER OF DTP SHEET (third parameter of user input)
stop_tokens = ui.split(' ')
row_num = int(stop_tokens[2]) - 1

# GET THE ROW
row = self.google_sheet.sheet[row_num]
if len(stop_tokens) == 4:
    quantity = int(stop_tokens[3])
else:
    quantity = int(row[4])
    
# DETERMINE STOP ORDER PARAMETERS
side = 'sell' if (
        row[3].upper() == 'LONG'
        or row[3].upper() == 'BUY'
        or row[3].upper() == 'B') else 'buy'
if side == 'sell': quantity *= -1
symbol = row[2].split()[0].upper()
if len(row) >= 13: stop_price = round(float(row[12]), 2)
account = row[1].upper()
if side == 'sell':
    stop_limit = round(stop_price * .99, 2)
else:
    stop_limit = round(stop_price * 1.01, 2)

# DETERMINE TIME STOP PARAMETERS
# works if it's before 1
# cancels and replaces itself every x minutes with 100 less shares until it reaches 0
now = datetime.datetime.now()
now_time = now.time()
if now_time >= datetime.time(13, 0):
    self.print2('took late for time stop')
    return 0

# TIME TO 1PM:
elapse = datetime.datetime.combine(datetime.date.today(), datetime.time(13, 0)) - now

# HOW MANY SECONDS TO SLEEP BEFORE CANCEL AND REPLACE WITH 100 LESS SHARES
sleep_seconds = int(elapse.total_seconds() / (abs(quantity) / 100))

# CALL time_stop FUNCTION ON ANOTHER THREAD
self.threads.append(Thread(target=self.time_stop,
                           args=((sleep_seconds, quantity, symbol, stop_price, stop_limit, account,))))
self.threads[len(self.threads) - 1].start()

self.print2('thread called')


# Function is run on its own thread
# Cancels and replaces with 100 less shares every sleep_seconds seconds
# sleep_seconds : How much to wait between cancel replaces
def time_stop(self,sleep_seconds, quantity, symbol, stop_price, stop_limit, account):
    # sleep time, quantity, symbol, stop price, stop limit, account)
    # submit first order outside the loop so stop is out there immediately
    o = self.of.generate_stop_limit_order(quantity, symbol, stop_price, stop_limit, account)
    self.send_order(o)
    
    while True:
        # SLEEP
        time.sleep(sleep_seconds)
        
        # ADJUST QUANTITY FOR NEW ORDER IF MORE THAN 100 SHARE LEFT
        if quantity > 100:
            quantity -= 100
        elif quantity < -100:
            quantity += 100
        else:
            # IF LESS THAN OR EQUAL TO 100 SHARES CANCEL AND STOP
            self.em.cancel_order(o)
            break
        
        # IF THE ORDER IS EXECUTED OR PARTIALLY EXECUTED STOP
        if o.status == order_status_type.executed: break
        if o.status == order_status_type.partial_open: break
        if o.status == order_status_type.canceled:break
            
        # CANCEL 
        self.em.cancel_order(o)
        while o.status != order_status_type.canceled:
            time.sleep(.1)
        # REPLACE
        o = self.of.generate_stop_limit_order(quantity, symbol, stop_price, stop_limit, account)
        self.send_order(o)
        
    self.print2 ('\nthread done\ninteractive>')